In [23]:
from pathlib import Path

import numpy as np
from scipy import stats
import shapely
import shapely.geometry
from cartopy.io import shapereader

In [16]:
# https://math.stackexchange.com/q/444700/593484
# See alternative method 1: https://corysimon.github.io/articles/uniformdistn-on-sphere/
NUM_WORLD_LOCATIONS = 37067
# in miles:
MEAN_RADIUS_OF_EARTH = 3958.7613
DATA_PATH = Path("../data/")


In [ ]:

vecs_on_sphere = np.empty(shape=(3, NUM_WORLD_LOCATIONS))
# Loop as many times as we want to get a vector
# on the unit-sphere.
for i in range(NUM_WORLD_LOCATIONS):
    # Try to get a vector long enough so
    # that it is easy to rescale it
    # to land on the unit-sphere.
    
    # Before loop
    norm_of_v = 0
    while (norm_of_v < 0.0001):
        # We have to try getting more numerically
        # stable random normal deviates.
        x = stats.norm.rvs(size=1).item()
        y = stats.norm.rvs(size=1).item()
        z = stats.norm.rvs(size=1).item()
        
        norm_of_v = np.sqrt(x**2 + y**2 + z**2)

    # We found a long enough vector that can now
    # be stretched so that it lands on the unit-sphere
    # without any numerical problems.
    vecs_on_sphere[0, i] = x / norm_of_v
    vecs_on_sphere[1, i] = y / norm_of_v
    vecs_on_sphere[2, i] = z / norm_of_v
    


In [8]:
# https://en.m.wikipedia.org/wiki/Great-circle_distance
# Before loop:
# Fix a point on the unit-sphere.
fixed_point = vecs_on_sphere[:, 0]

delta_sigma = np.empty(shape=vecs_on_sphere.shape[1] - 1)
# Get a vector of greatest-circle
# distances to the other points.
for j in range(1, vecs_on_sphere.shape[1]):
    delta_sigma[j - 1] = np.arctan2(
        np.linalg.norm(np.cross(fixed_point, vecs_on_sphere[:, j]), ord=2),
        np.dot(fixed_point, vecs_on_sphere[:, j])
    )

In [9]:
# Now, scale to the size of the Earth.
distances = MEAN_RADIUS_OF_EARTH * np.abs(delta_sigma)

In [11]:
distances.sort()
delta_sigma.sort()

In [12]:
distances[NUM_WORLD_LOCATIONS - 10: ]

array([12290.86136627, 12307.2787999 , 12311.68616744, 12314.81963336,
       12316.77920186, 12322.82925048, 12338.33403214, 12371.92356364,
       12387.32573402])

In [20]:
# https://github.com/SciTools/cartopy/issues/1325
cartopy.config["data_dir"] = Path(DATA_PATH, "raw_data")

In [21]:
# https://datascience.stackexchange.com/questions/112104/sampling-from-earths-landmass
natural_earth_data = shapereader.natural_earth(
    resolution="10m",
    category="physical",
    name="land"
)

/home/justin/bin/mambaforge/envs/doe_group_project/lib/python3.11/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


In [24]:
print(natural_earth_data)

../data/raw_data/shapefiles/natural_earth/physical/ne_10m_land.shp


In [37]:
land_on_earth = shapely.union_all(
    geometries=list(shapereader.Reader(natural_earth_data).geometries())
)

shapely.prepare(
    geometry=land_on_earth
)

In [39]:
dir(land_on_earth)

['__and__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__geo_interface__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__xor__',
 '_geom',
 '_geom_prepared',
 '_ndim',
 '_repr_svg_',
 'almost_equals',
 'area',
 'boundary',
 'bounds',
 'buffer',
 'centroid',
 'contains',
 'contains_properly',
 'convex_hull',
 'coords',
 'covered_by',
 'covers',
 'crosses',
 'difference',
 'disjoint',
 'distance',
 'dwithin',
 'envelope',
 'equals',
 'equals_exact',
 'geom_type',
 'geometryType',
 'geoms',
 'has_z',
 'hausdorff_distance',
 'interpolate',
 'intersection',
 'intersects',
 'is_closed',
 'is_empty',
 'is_ring',
 'is_simple',
 'is_valid',
 '

In [42]:
land_on_earth.contains(shapely.geometry.Point(31.144318, -78.837661))

True